This code is only used for the coursera capstone project

# First Assignment - Capstone Project Notebook

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Second assignment - Segmenting and Clustering Neighborhoods in Toronto

## Part One - read wikipage into dataframe

In [58]:
import urllib

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup
import bs4 as bs

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1937
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1937
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -

In [37]:
# use pandas to read wikipage
url_wiki = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
table_wiki = pd.read_html(url_wiki, header=[0])
pd_table_wiki = table_wiki[0]
pd_table_wiki.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [39]:
#rename colums 
Toronto_neighborhoods = pd_table_wiki.rename(index=str, columns = {'Postal Code':'PostalCode','Neighbourhood':'Neighborhood'})

# ignore the 'not assigned' in the 'Boroug' column
Toronto_neighborhoods = Toronto_neighborhoods[Toronto_neighborhoods['Borough'] !='Not assigned']
Toronto_neighborhoods.reset_index(drop=True,inplace=True)
Toronto_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
# group the items with the same postal code into one row, separate the neigborhoods by commas
Toronto_neighborhoods = Toronto_neighborhoods.groupby('PostalCode', as_index='False').agg(lambda x: ','.join(set(x.dropna())))
Toronto_neighborhoods.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [41]:
# if a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough
Toronto_neighborhoods.loc[Toronto_neighborhoods['Neighborhood'] == 'Not assigned', 'Neighborhood'] = Toronto_neighborhoods['Borough']
Toronto_neighborhoods.head()

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [42]:
# print out the shape of the dataframe
Toronto_neighborhoods.shape


(103, 2)

## Part two - Geocoder

In [43]:
!conda install -c conda-forge geocoder --yes 
import geocoder

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0      

    jupyterlab_server-1.2.0    |             py_0          25 KB  conda-forge
    jupyterlab_widgets-1.0.0   |     pyhd8ed1ab_1         130 KB  conda-forge
    jxrlib-1.1                 |       h7f98852_2         235 KB  conda-forge
    keras-applications-1.0.8   |             py_1          30 KB  conda-forge
    keras-preprocessing-1.1.2  |     pyhd8ed1ab_0          34 KB  conda-forge
    keyring-18.0.0             |           py37_0          50 KB  conda-forge
    kiwisolver-1.3.1           |   py37h2527ec5_1          78 KB  conda-forge
    krb5-1.17.2                |       h926e7f8_0         1.4 MB  conda-forge
    lcms2-2.12                 |       hddcbb42_0         443 KB  conda-forge
    ld_impl_linux-64-2.35.1    |       hea4e1c9_2         618 KB  conda-forge
    lerc-2.2.1                 |       h9c3ff4c_0         213 KB  conda-forge
    libaec-1.0.4               |       h9c3ff4c_1          31 KB  conda-forge
    libblas-3.9.0              |            8_mkl          12 KB

expat-2.2.10         | 164 KB    | ##################################### | 100% 
webencodings-0.5.1   | 12 KB     | ##################################### | 100% 
mpfr-4.0.2           | 648 KB    | ##################################### | 100% 
async_generator-1.10 | 18 KB     | ##################################### | 100% 
pthread-stubs-0.4    | 5 KB      | ##################################### | 100% 
libclang-11.1.0      | 19.2 MB   | ##################################### | 100% 
pyqt-impl-5.12.3     | 5.9 MB    | ##################################### | 100% 
tifffile-2021.3.5    | 127 KB    | ##################################### | 100% 
pandas-1.2.3         | 11.8 MB   | ##################################### | 100% 
typing_extensions-3. | 25 KB     | ##################################### | 100% 
libtiff-4.2.0        | 633 KB    | ##################################### | 100% 
google-auth-oauthlib | 19 KB     | ##################################### | 100% 
libedit-3.1.20191231 | 121 K

pyasn1-0.4.8         | 53 KB     | ##################################### | 100% 
testpath-0.4.4       | 85 KB     | ##################################### | 100% 
libdeflate-1.7       | 67 KB     | ##################################### | 100% 
s3transfer-0.3.4     | 51 KB     | ##################################### | 100% 
backports-1.0        | 4 KB      | ##################################### | 100% 
ptyprocess-0.7.0     | 16 KB     | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
grpcio-1.36.1        | 2.1 MB    | ##################################### | 100% 
ipykernel-5.5.0      | 166 KB    | ##################################### | 100% 
libxml2-2.9.10       | 1.3 MB    | ##################################### | 100% 
libgcc-ng-9.3.0      | 7.8 MB    | ##################################### | 100% 
retrying-1.3.3       | 11 KB     | ##################################### | 100% 
pyparsing-2.4.7      | 60 KB

zipp-3.4.1           | 11 KB     | ##################################### | 100% 
yaml-0.2.5           | 82 KB     | ##################################### | 100% 
nest-asyncio-1.4.3   | 9 KB      | ##################################### | 100% 
chardet-4.0.0        | 204 KB    | ##################################### | 100% 
wrapt-1.12.1         | 47 KB     | ##################################### | 100% 
libwebp-base-1.2.0   | 815 KB    | ##################################### | 100% 
scikit-image-0.18.1  | 11.5 MB   | ##################################### | 100% 
pyqt-5.12.3          | 21 KB     | ##################################### | 100% 
werkzeug-1.0.1       | 239 KB    | ##################################### | 100% 
python_abi-3.7       | 4 KB      | ##################################### | 100% 
cx_oracle-8.1.0      | 193 KB    | ##################################### | 100% 
jxrlib-1.1           | 235 KB    | ##################################### | 100% 
libcblas-3.9.0       | 11 KB

done


In [49]:
# read in csv file with geo information and save it in a new dataframe
Toronto_geo = pd.read_csv('https://cocl.us/Geospatial_data')
Toronto_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
# rename colum so it fits to the toronto data
Toronto_geo.columns = ['PostalCode','Latitude','Longitude']
Toronto_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
# merge toronto map with the geo data
toronto_with_geo = pd.merge(left=Toronto_neighborhoods, right=Toronto_geo, on='PostalCode')
toronto_with_geo.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part three - Explore and cluster the neigborhoods of Toronto

In [54]:
# focus of the analysis is on North York
address = 'North York'
geolocator = Nominatim(user_agent="northyork_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [100]:
# Filter the data of North York
northyork_data = toronto_with_geo[toronto_with_geo['Borough'] == 'North York'].reset_index(drop=True)
northyork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [59]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Borough'], northyork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [60]:
CLIENT_ID = '30ZISF4NJK3DTS5WVFADNIHF3NYBZ10ORELR1YM5HLUQHI5N' # your Foursquare ID
CLIENT_SECRET = 'FPA2PNFUOCL2H4B5UL2QW5ZGKYSWHSHP2PIQTEORAQOHHTOA' # your Foursquare Secret
ACCESS_TOKEN = 'PHCAY2RGUINL5GIS2NCR2HPFVDS5CGGBVCTDTZJQ3TSBVKL4' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 30ZISF4NJK3DTS5WVFADNIHF3NYBZ10ORELR1YM5HLUQHI5N
CLIENT_SECRET:FPA2PNFUOCL2H4B5UL2QW5ZGKYSWHSHP2PIQTEORAQOHHTOA


In [101]:
# Explore venues top 100 venues in 500m radius. Use coordinates of first neighborhood
LIMIT = 100
radius = 500
northyork_latitude = northyork_data['Latitude'][0]
northyork_longitude = northyork_data['Longitude'][0]
VERSION = '20180605'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, northyork_latitude, northyork_longitude, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60511ae2222e3f1aec5f8367'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d8771f6651041bd5e499b30',
       'name': 'New York Fries',
       'location': {'address': '1800 Sheppard Avenue East',
        'crossStreet': 'in Fairview Mall',
        'lat': 43.80366383775661,
        'lng': -79.36390544874158,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80366383775661,
          'lng': -79.36390544874158}],
        'distance': 38,
    

In [102]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [103]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()
# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,New York Fries,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",43.803664,-79.363905
1,Eagle's Nest Golf Club,"[{'id': '4bf58dd8d48988d1e6941735', 'name': 'G...",43.805455,-79.364186
2,AY Jackson Pool,"[{'id': '4bf58dd8d48988d15e941735', 'name': 'P...",43.804515,-79.366138
3,Villa Madina,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",43.801685,-79.363938
4,Duncan Creek Park,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D...",43.805539,-79.360695


In [104]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [105]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [106]:
northyork_venues = getNearbyVenues(names=northyork_data['Neighborhood'], latitudes=northyork_data['Latitude'], longitudes=northyork_data['Longitude'])


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


In [107]:
# only show unique venues
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))


There are 102 uniques categories.


In [108]:
# Amount of 
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Don Mills,26,26,26,26,26,26
Downsview,17,17,17,17,17,17
"Fairview, Henry Farm, Oriole",70,70,70,70,70,70
Glencairn,4,4,4,4,4,4
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


In [109]:
# Analyze each neighborhood

# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
northyork_onehot.shape

(253, 103)

In [111]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,...,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.038462,0.000000,0.000000,...,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.014286,0.000000,0.000000,0.014286,0.000000,0.028571,0.028571,...,0.014286,0.000000,0.014286,0.000000,0.014286,0.014286,0.0,0.014286,0.000000,0.028571
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [112]:
northyork_grouped.shape

(19, 103)

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,Gift Shop,Deli / Bodega,Ice Cream Shop
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Indian Restaurant,Pizza Place,Pub,Pharmacy,Restaurant,Liquor Store,Grocery Store
3,Don Mills,Gym,Coffee Shop,Beer Store,Restaurant,Dim Sum Restaurant,Caribbean Restaurant,Café,Japanese Restaurant,Italian Restaurant,Bike Shop
4,Downsview,Grocery Store,Park,Baseball Field,Electronics Store,Discount Store,Hotel,Liquor Store,Business Service,Shopping Mall,Snack Place


In [115]:
#Cluster neighborhoods

# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 3], dtype=int32)

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Pool,Fast Food Restaurant,Dog Run,Mediterranean Restaurant,Women's Store,Deli / Bodega,Chocolate Shop,Clothing Store,Coffee Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Japanese Restaurant,Mobile Phone Shop,Juice Bar,Bank,Electronics Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,2.0,Park,Dessert Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
